In [6]:
%pip install python-dotenv tavily-python requests langchain faiss-cpu sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import json
import requests
import time
from tavily import TavilyClient
from dotenv import load_dotenv
from IPython.display import display, Markdown

# LangChain components for RAG
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

# --- Load API Keys & Configure Clients (same as before) ---
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
OPENROUTER_URL = "https://openrouter.ai/api/v1/chat/completions"
OPENROUTER_HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "https://github.com/guy-hartstein/company-research-agent",
    "X-Title": "Wavess GTM Intelligence Prototype",
}

# --- Define LLMs & Embedding Model ---
LLM_MODELS = [
    "google/gemini-2.0-flash-exp:free",
    "kwaipilot/kat-coder-pro:free",
    "openrouter/polaris-alpha",
]

# Use a powerful, free, and local embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/var/folders/fy/l_bc5j753tx9_bp8mg05l0x00000gn/T/ipykernel_51645/4166252925.py:37: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [8]:
def call_openrouter_with_retry(prompt):
    """
    Sends a prompt to OpenRouter, trying a list of models in sequence.
    """
    for model in LLM_MODELS:
        try:
            print(f"Attempting to call model: {model}...")
            response = requests.post(
                url=OPENROUTER_URL,
                headers=OPENROUTER_HEADERS,
                data=json.dumps({"model": model, "messages": [{"role": "user", "content": prompt}]})
            )
            response.raise_for_status()
            print(f"Successfully received response from {model}.")
            return response.json()['choices'][0]['message']['content']
        except requests.exceptions.RequestException as e:
            print(f"Failed to call {model}: {e}")
            print("Waiting 5 seconds before trying the next model...")
            time.sleep(5)
    print("FATAL: All models failed.")
    return None

In [9]:
def run_collector_and_build_vectorstore(company):
    """
    Step 1: Runs targeted searches, processes the content, and builds an in-memory
    vector store (our RAG knowledge base).
    """
    print("Step 1/3: Running targeted searches and building knowledge base...")
    gtm_queries = [
        f'"{company}" recent major partnerships with enterprise companies',
        f'"{company}" new CFO OR C-level executive hires',
        f'"{company}" latest funding round and valuation',
        f'"{company}" new product launches related to AI, orchestration, or enterprise',
        f'"{company}" strategic shift OR focus on new industries',
    ]
    
    # 1. Collect
    all_results = []
    for query in gtm_queries:
        print(f"  - Searching for: {query}")
        search_results = tavily_client.search(query=query, search_depth="advanced", max_results=3)['results']
        all_results.extend(search_results)
    print(f"Collected {len(all_results)} total search results.")

    # 2. Chunk
    # Convert search results into LangChain Document objects
    docs = [Document(page_content=res['content'], metadata={'source': res['url']}) for res in all_results]
    
    # Split the documents into smaller, more manageable chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunked_docs = text_splitter.split_documents(docs)
    print(f"Split content into {len(chunked_docs)} chunks.")

    # 3. Embed & Store
    # This creates a searchable vector database in memory from the document chunks
    vector_store = FAISS.from_documents(chunked_docs, embedding_model)
    print("Knowledge base created successfully.")
    
    return vector_store.as_retriever()


def run_rag_analyst_node(retriever, company):
    """
    Step 2: Asks key GTM questions one by one, retrieving relevant context
    from the vector store for each question before generating an answer.
    """
    print("\nStep 2/3: Answering key GTM questions using RAG...")
    
    key_gtm_questions = [
        f"What is {company}'s most recently reported valuation and funding status?",
        f"What major new enterprise partnerships or customers has {company} announced?",
        f"Who are the recent, significant C-level executive hires at {company}?",
        f"What major new products has {company} launched, especially regarding AI, Orchestration, or Enterprise features?",
        f"What are the key strategic shifts or new industry focuses for {company}?"
    ]
    
    grounded_answers = []
    for question in key_gtm_questions:
        print(f"  - Answering question: {question}")
        
        # 1. Retrieve
        relevant_docs = retriever.get_relevant_documents(question)
        context_str = "\n\n".join([f"Source: {doc.metadata['source']}\nContent: {doc.page_content}" for doc in relevant_docs])
        
        # 2. Generate
        prompt = f"""
        You are a factual analyst. Answer the user's question based *only* on the provided context.
        List the key facts and cite the source URL for each fact.

        CONTEXT:
        ---
        {context_str}
        ---
        
        QUESTION: {question}

        ANSWER:
        """
        answer = call_openrouter_with_retry(prompt)
        if answer:
            grounded_answers.append({"question": question, "answer": answer})
        time.sleep(3) # Small delay between questions
        
    print("All key questions answered.")
    return grounded_answers


def run_editor_node(grounded_answers, company):
    """
    Step 3: Assembles the high-quality, pre-answered questions into the final
    polished GTM report.
    """
    print("\nStep 3/3: Compiling the final GTM report...")
    answers_str = json.dumps(grounded_answers, indent=2)
    prompt = f"""
    You are a Senior Go-To-Market (GTM) Analyst. Your task is to synthesize the following pre-answered, fact-checked research notes into a final, polished intelligence briefing about "{company}".

    **Instructions:**
    1.  Review the questions and their detailed answers below.
    2.  Do not add any new information. Your job is to structure and summarize.
    3.  Create a comprehensive report in Markdown that includes:
        a. A concise **Executive Summary**.
        b. A **Key GTM Signals** section, summarizing the most important findings from the notes.
        c. An **Actionable GTM Recommendations** section for a sales/marketing team based on the findings.
        d. A **Reference List** containing all the unique source URLs mentioned in the answers.

    **RESEARCH NOTES:**
    ---
    {answers_str}
    ---
    
    **FINAL GTM INTELLIGENCE REPORT:**
    """
    return call_openrouter_with_retry(prompt)

In [10]:
# --- Define the Research Target ---
COMPANY_NAME = "Stripe"

# --- Run the RAG-Powered Workflow ---
print("--- Starting GTM Intelligence Report Generation (RAG Version) ---")

# Step 1: Collect data and build the searchable knowledge base
retriever = run_collector_and_build_vectorstore(COMPANY_NAME)

# Step 2: Use RAG to answer key GTM questions factually
if retriever:
    grounded_answers = run_rag_analyst_node(retriever, COMPANY_NAME)
else:
    grounded_answers = []

# Step 3: Compile the final report from the high-quality answers
if grounded_answers:
    final_report = run_editor_node(grounded_answers, COMPANY_NAME)
    print("\n--- Report Generation Complete! ---")
else:
    final_report = "Could not generate a report as no grounded answers were available."

# --- Display the Final GTM Intelligence Report ---
display(Markdown(final_report))

--- Starting GTM Intelligence Report Generation (RAG Version) ---
Step 1/3: Running targeted searches and building knowledge base...
  - Searching for: "Stripe" recent major partnerships with enterprise companies
  - Searching for: "Stripe" new CFO OR C-level executive hires
  - Searching for: "Stripe" latest funding round and valuation
  - Searching for: "Stripe" new product launches related to AI, orchestration, or enterprise
  - Searching for: "Stripe" strategic shift OR focus on new industries
Collected 15 total search results.
Split content into 23 chunks.
Knowledge base created successfully.

Step 2/3: Answering key GTM questions using RAG...
  - Answering question: What is Stripe's most recently reported valuation and funding status?
Attempting to call model: google/gemini-2.0-flash-exp:free...


/var/folders/fy/l_bc5j753tx9_bp8mg05l0x00000gn/T/ipykernel_51645/2488266599.py:60: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(question)


Failed to call google/gemini-2.0-flash-exp:free: 429 Client Error: Too Many Requests for url: https://openrouter.ai/api/v1/chat/completions
Waiting 5 seconds before trying the next model...
Attempting to call model: kwaipilot/kat-coder-pro:free...
Failed to call kwaipilot/kat-coder-pro:free: 429 Client Error: Too Many Requests for url: https://openrouter.ai/api/v1/chat/completions
Waiting 5 seconds before trying the next model...
Attempting to call model: openrouter/polaris-alpha...
Successfully received response from openrouter/polaris-alpha.
  - Answering question: What major new enterprise partnerships or customers has Stripe announced?
Attempting to call model: google/gemini-2.0-flash-exp:free...
Failed to call google/gemini-2.0-flash-exp:free: 429 Client Error: Too Many Requests for url: https://openrouter.ai/api/v1/chat/completions
Waiting 5 seconds before trying the next model...
Attempting to call model: kwaipilot/kat-coder-pro:free...
Failed to call kwaipilot/kat-coder-pro:fre

## Executive Summary

Stripe is a $95B payments and financial infrastructure leader that is accelerating into an enterprise- and AI-first strategy. Recent funding ($6.5B Series I), high-profile enterprise wins (including Nvidia’s migration to Stripe Billing), extensive penetration of Fortune 100 and AI-native companies, and a broadened product stack (AI foundation model for payments, orchestration, stablecoin accounts, and the Agentic Commerce Protocol) reinforce Stripe’s position as a critical, scalable “financial OS” for modern businesses. These moves signal a platform built to lock in large customers, orchestrate complex multi-PSP environments, enable AI-driven and agent-led commerce, and prepare for public-market scale—making Stripe both a formidable competitor and a highly strategic ecosystem partner.

---

## Key GTM Signals

### 1. Financial Strength & Strategic Maturity
- **Valuation:** $95B.
- **Funding:** $6.5B Series I led by top-tier investors (Thrive Capital, Andreessen Horowitz, Sequoia, General Catalyst).
- **Signal:** Reinforces market confidence, supports long-term infrastructure investments, and suggests readiness for scaled enterprise commitments and potential IPO trajectory.

### 2. Enterprise Penetration & Customer Proof
- **Customer Base:**
  - Serves **half of the Fortune 100**.
  - Serves **78% of firms on the Forbes AI 50**, including OpenAI and Anthropic.
  - Named enterprise customers include **NVIDIA, Pepsi, Rivian, Cloudflare, AMC Networks, Forbes**.
- **Signal:** Strong validation as trusted infrastructure for global, high-volume, and AI-native enterprises; rising switching costs and reference power in complex, regulated, and high-growth environments.

### 3. Strategic Enterprise & AI Partnerships
- **Nvidia Partnership:** Deeper collaboration; Nvidia migrated its entire subscriber base to Stripe Billing.
- **ACP with OpenAI & PwC:** Co-developed Agentic Commerce Protocol plus PwC partnership for enterprise adoption.
- **Signal:** Stripe is positioning at the intersection of AI, agents, and payments—becoming core to how AI-native and enterprise ecosystems transact.

### 4. Product & Platform Expansion

**a. AI & Risk/Performance**
- Launched an **AI foundation model for payments** to improve decisions, optimization, and fraud/risk outcomes.
- Signal: Differentiates on intelligence and performance, not just basic processing.

**b. Orchestration**
- Introduced **Orchestration** to manage multiple payment providers via one Stripe dashboard, including for non-Stripe processing.
- Signal: Stripe is embedding itself as the control layer in complex payment stacks—owning the customer relationship even in multi-PSP environments.

**c. Stablecoin-Powered Accounts**
- Launched **stablecoin-powered accounts**.
- Signal: Aligns with next-gen payment rails and global, digital-native treasury needs—appealing to high-velocity and cross-border businesses.

**d. Full-Stack Fintech Positioning**
- Continues to expand beyond payments into a **comprehensive fintech stack**: fraud prevention, analytics, lending, Billing, Issuing, Treasury, and more.
- Signal: Strategy to increase product density and stickiness; positions Stripe as a one-stop financial infrastructure provider.

### 5. Leadership & IPO Readiness
- **New CFO:** Steffan Tomlinson (ex-Confluent, experience at Google and other large tech firms).
- **Signal:** Explicit move toward operational rigor, scalability, and capital markets readiness—consistent with IPO expectations and enterprise-grade governance.

### 6. Core Strategic Positioning
- **“Financial plumbing” for platforms:** Deep embedding into ecosystems (e.g., platforms like Shopify, Lyft) where Stripe grows as they grow.
- **Enterprise focus:** Increasing emphasis on large, complex customers and AI-first companies.
- **Signal:** Stripe is solidifying its role as indispensable infrastructure for both platforms and enterprises, raising barriers to entry for competitors.

---

## Actionable GTM Recommendations

### 1. Target Segments & Positioning

1. **Focus on enterprise and high-growth scale-ups**
   - Prioritize accounts that:
     - Operate globally and need multi-PSP, multi-currency, or complex billing.
     - Are AI-native, subscription-based, marketplace, or platform businesses.
   - Position against Stripe by emphasizing:
     - Differentiated capabilities where relevant (e.g., vertical specialization, bespoke compliance, local payment methods, service quality, or cost transparency).
     - Interoperability and reduced lock-in versus Stripe’s increasingly integrated stack.

2. **Lean into AI-native and agentic commerce narratives**
   - For AI, SaaS, and platform prospects:
     - Frame offerings as complementary or competitive to Stripe’s AI foundation model and ACP approach.
     - Highlight ability to integrate with AI agents, automate financial workflows, or enhance risk/authorization performance.

### 2. Competitive Displacement & Coexistence Plays

1. **Exploit Orchestration as an entry point**
   - Where Stripe Orchestration is present or under evaluation:
     - Offer value as a specialized provider inside multi-PSP architectures (e.g., better performance in certain geos, payment methods, or risk profiles).
     - Provide migration and optimization services that work alongside or in place of Stripe, focusing on reducing dependency.

2. **Target pain from one-stop-stack lock-in**
   - Speak to enterprises wary of:
     - Vendor concentration risk.
     - Limited customization or commercial leverage.
   - Provide GTM content and sales plays around:
     - Multi-provider resilience.
     - Transparent economics.
     - Regulatory and data localization flexibility versus a single global stack.

### 3. Enterprise Sales Motions

1. **Use Stripe’s references as proof of category maturity**
   - Acknowledge Stripe’s role to validate the category, then:
     - Position your solution as better suited for specific needs (regulated industries, niche markets, on-prem/hybrid, sovereignty, deep integration).
   - Build case studies that mirror Stripe-style logos (large platforms, AI leaders) to de-risk choice.

2. **Engage finance and ops stakeholders early**
   - Stripe’s CFO-level maturity and IPO trajectory imply buyers expect:
     - Robust reporting, compliance, auditability, cost control.
   - Arm sales teams with:
     - Detailed TCO models.
     - ROI calculators tied to approval uplift, churn reduction, working capital improvements.

### 4. Product Marketing & Messaging

1. **Theme: “Intelligent, open, resilient payments infrastructure”**
   - Emphasize:
     - Intelligence (risk/AI capabilities).
     - Openness (no lock-in, easy switching, ecosystem integrability).
     - Resilience (multi-rail, multi-PSP, regional redundancy).

2. **Content Strategy**
   - Publish:
     - Comparisons that articulate when enterprises should consider multi-PSP or alternatives to a monolithic provider.
     - POVs on AI in payments, stablecoins, and agentic commerce that reference Stripe’s moves as validation without adding new facts.

### 5. Partner & Ecosystem Strategy

1. **Collaborate with consultants & SIs**
   - Mirror Stripe’s collaboration with PwC:
     - Enable consultancies and SIs with playbooks for designing multi-PSP or specialized stacks including (or competing with) Stripe.
   - Train partners to:
     - Identify when Stripe’s integrated approach may not be optimal and propose complementary/alternative architectures.

---

## Reference List

- https://salestools.io/report/stripe-6-5b-series-i-july-2025  
- https://www.linkedin.com/posts/maryannazevedo_stripe-unveils-ai-foundation-model-for-payments-activity-7325996173222105088-Jr6z  
- https://rfadvisorslimited.com/stripe-unveils-ai-foundation-model-for-payments-reveals-deeper-partnership-with-nvidia-techcrunch/  
- https://fintechmagazine.com/news/payments-giant-stripe-nabs-10th-in-top-100-fintech-companies  
- https://stripe.com/newsroom/news/tour-newyork-2024  
- https://www.paymentsdive.com/news/former-google-cloud-cfo-heads-to-stripe/689910/  
- https://dataconomy.com/2025/05/08/stripe-dives-into-stablecoins-rolls-out-major-ai-tools/  
- https://www.pwc.com/us/en/about-us/newsroom/press-releases/stripe-collaboration-next-era-agentic-commerce.html  
- https://strategybreakdowns.com/p/stripe-platform-strategy  
- https://www.forbes.com/sites/danielwebber/2025/09/23/stripes-ipo-potential-is-stronger-than-ever/

In [11]:
# --- Save the final report to a Markdown file ---

# Define the filename
output_filename = "gtm_intelligence_report_stripe.md"

try:
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(final_report)
    print(f"Successfully saved the GTM report to: {output_filename}")
except Exception as e:
    print(f"An error occurred while saving the file: {e}")

Successfully saved the GTM report to: gtm_intelligence_report_stripe.md
